In [108]:
import pandas as pd
import numpy as np
import openai
import os
import gspread
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAIKEY')


RAW_FOLDER = "D:/Dropbox (Personal)/Personal Work/_Projects2023/01_city-never-was/_data/_raw/_city_profiles"
TRANSFORM_FOLDER = "D:/Dropbox (Personal)/Personal Work/_Projects2023/01_city-never-was/_data/_transformed/t_city_profiles"
if not os.path.exists(TRANSFORM_FOLDER):
    os.makedirs(TRANSFORM_FOLDER)
FILENAME = "r_city_gdp_wikipidia.xlsx"

df = pd.read_excel(f"{RAW_FOLDER}/{FILENAME}")
df["pop_metro"] = df['pop_metro'].apply(lambda x: float(x.replace(",", "").strip()) if type(x) == str else x)
df['gdp_billion'] = df['gdp_billion'].apply(lambda x: float(x.replace(",", "").strip()) if type(x) == str else x)

In [9]:
import os
import gspread
import numpy as np
import pandas as pd

def read_url(url, SHEET_NAME):
    SHEET_ID = url.split("/")[5]
    spreadsheet = gc.open_by_key(SHEET_ID)
    worksheet = spreadsheet.worksheet(SHEET_NAME)
    rows = worksheet.get_all_records()
    df_spread = pd.DataFrame(rows)
    return df_spread, worksheet

serviceaccount = "../../google_drive_personal.json"
gc = gspread.service_account(filename=serviceaccount)
GC_URL = "https://docs.google.com/spreadsheets/d/1o5gFmZPUoDwrrbfE6M26uJF3HnEZll02ivnOxP6K6Xw/edit?usp=sharing"

SHEETNAME = "select_city_classifier"
city_meta, other_worksheet = read_url(GC_URL, SHEETNAME)
city_meta = city_meta[city_meta['City']!=''].reset_index(drop = True)
city_meta["country_clean"] = np.where(city_meta["Country"].isin(["USA", "United States"]), 
                                      "United States of America",
                                      city_meta["Country"]
                                     )




In [103]:
city_list_1 = city_meta['City'].tolist()
city_list_2 = df['Metro'].tolist()

In [109]:
def find_city(city, city_list):
    
    for c in city_list:
        if city.lower().replace(" ", "") in c.lower().replace(" ", "").replace("-",""):
            return c
    return None

In [110]:
mapped = []
for city in city_list_1:
    if city =='Sao Paulo':
        mapped.append('Greater São Paulo')
    elif city =='Portland, OR':
        mapped.append('Portland-South Portland, ME MSA')
    else:
        mapped.append(find_city(city, city_list_2))
    

In [116]:
df_results = pd.DataFrame({"City": city_list_1, "city_wiki": mapped})
df_results = city_meta[["City","country_clean",'State/Province','urban_pop']].merge(df_results, on = 'City')\
    .merge(df[['Metro','gdp_billion', 'pop_metro','source_year']], 
                 left_on = 'city_wiki', 
                 right_on = 'Metro', how = 'left').drop("city_wiki", axis = 1)\
                     .rename(columns = {'gdp_billion':'gdp_billion_metro','pop_metro':'gdp_pop_metro'})
                     
df_results['gdp_per_cap'] = df_results['gdp_billion_metro'].astype(float)/df_results['gdp_pop_metro']*1_000_000_000

In [117]:
df_results.to_csv(os.path.join(TRANSFORM_FOLDER, "t_city_gdp_mapping.csv"), index = False)

: 

# Get Country level gdp per capita to fill in the nan values

In [27]:
import pandas as pd
import os
TRANSFORM_FOLDER = "D:/Dropbox (Personal)/Personal Work/_Projects2023/01_city-never-was/_data/_transformed/t_city_profiles"
CURATED_FOLDER = "D:/Dropbox (Personal)/Personal Work/_Projects2023/01_city-never-was/_data/_curated/city_profiles"
RAW_FOLDER = "D:/Dropbox (Personal)/Personal Work/_commondata/_world_data/03_global_gdp"
if not os.path.exists(CURATED_FOLDER):
    os.makedirs(CURATED_FOLDER)

FILENAME = "imf-dm-export-20240514.xlsx"

In [ ]:
df = pd.read_excel(f"{RAW_FOLDER}/{FILENAME}").rename(columns = {"GDP per capita, current prices\n (U.S. dollars per capita)":"country"})
df.columns = [str(x) for x in df.columns]
df = df[["country", "2022"]].rename(columns ={ "2022":"gdp_per_cap_country_2022"})


country_name_mapping = {
    'United States':'United States of America',
    'Türkiye, Republic of':'Turkey',
    'Korea, Republic of': 'Republic of Korea',
}
df['admin_mapped'] = df['country'].apply(lambda x: country_name_mapping[x] if x in country_name_mapping.keys() else x)

df_stage1 = pd.read_csv(os.path.join(TRANSFORM_FOLDER, "t_city_gdp_mapping.csv"))
df_stage1['admin_mapped'] = np.where(
    df_stage1['City']=='Taipei',
    'Taiwan Province of China',
    df_stage1['country_clean']
)
df_stage1['admin_mapped'] = np.where(
    df_stage1['City']=='Hong Kong',
    'Hong Kong SAR',
    df_stage1['admin_mapped']
)
df_stage1.head()

In [25]:
df_stage2 = df_stage1.merge(df, on = 'admin_mapped', how = 'left')
df_stage2[df_stage2['gdp_per_cap_country_2022'].isnull()]['country_clean'].unique()

array([], dtype=object)

In [28]:
df_stage2.drop(["admin_mapped", 'country'], axis = 1)\
    .to_csv(os.path.join(CURATED_FOLDER, "c_city_gdp.csv"), index = False)